In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("주식분석")

LangSmith 추적을 시작합니다.
[프로젝트명]
주식분석


In [ ]:
import requests
import zipfile
import io
import xml.etree.ElementTree as ET
import pandas as pd

# API 정보
api_url = "https://opendart.fss.or.kr/api/corpCode.xml"
api_key = "xxx"  # 자신의 API Key로 변경

# API 요청
response = requests.get(api_url, params={"crtfc_key": api_key})

# 응답 확인
if response.status_code == 200:
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        file_name = z.namelist()[0]  # ZIP 파일 내 XML 파일 이름
        with z.open(file_name) as xml_file:
            xml_content = xml_file.read().decode("utf-8")
            
            # XML 파싱
            root = ET.fromstring(xml_content)
            data = []
            for corp in root.findall("list"):
                corp_code = corp.find("corp_code").text
                corp_name = corp.find("corp_name").text
                corp_eng_name = corp.find("corp_eng_name").text
                stock_code = corp.find("stock_code").text
                modify_date = corp.find("modify_date").text
                
                data.append([corp_code, corp_name, corp_eng_name, stock_code, modify_date])
            
            # DataFrame 생성
            df = pd.DataFrame(data, columns=["고유번호", "정식명칭", "영문명칭", "종목코드", "최종변경일"])
            
            # 검색할 고유명칭 입력
            search_name = input("검색할 회사 이름을 입력하세요: ")
            
            # 고유명칭으로 검색
            result = df[df["정식명칭"].str.contains(search_name, case=False, na=False)]
            
            # 결과 출력
            if not result.empty:
                print("\n검색 결과:")
                print(result)
            else:
                print("\n해당 이름의 회사가 없습니다.")
else:
    print("API 호출 실패:", response.status_code)


검색할 회사 이름을 입력하세요:  솔트룩스



검색 결과:
           고유번호     정식명칭              영문명칭    종목코드     최종변경일
53990  01604159  솔트룩스벤처스  Saltlux Ventures          20211222
99934  01050738     솔트룩스       SALTLUX Inc  304100  20240328


In [ ]:
import requests
import pandas as pd

# 🔹 API 정보
api_url = "https://opendart.fss.or.kr/api/list.json"  # JSON 형식으로 요청
api_key = "xxx"  # 여기에 본인의 인증키 입력

# 🔹 요청 파라미터 설정
params = {
    "crtfc_key": api_key,  # API 인증키
    "corp_code": "01050738", # 고유번호(8자리)
    "bgn_de": "20230101",  # 조회 시작일 (예: 2024년 1월 1일)
    "end_de": "20250228",  # 조회 종료일 (예: 2024년 2월 19일)
    "last_reprt_at": "Y", # 최종보고서만 검색여부(Y OR N)
    "pblntf_ty": "A", # 공시유형(A:정기공시)
    "page_no": "1",  # 페이지 번호
    "page_count": "10"  # 페이지당 항목 수
}

# 🔹 API 호출
response = requests.get(api_url, params=params)

# 🔹 응답 확인
import requests
import pandas as pd

# 🔹 응답 확인 및 URL 생성
if response.status_code == 200:
    data = response.json()  # JSON 데이터 변환
    if "list" in data:
        df = pd.DataFrame(data["list"])  # DataFrame으로 변환
        
        # 🔹 rcept_no 사용하여 링크 생성
        df["URL"] = "https://dart.fss.or.kr/dsaf001/main.do?rcpNo=" + df["rcept_no"]

        #출력 옵션 설정
        pd.set_option('display.max_colwidth', None)
        
        # 🔹 데이터 출력 (링크 포함)
        print(df[["corp_name", "report_nm", "rcept_no", "URL"]].head())
    else:
        print("데이터 없음:", data)
else:
    print("API 호출 실패:", response.status_code)

  corp_name        report_nm        rcept_no  \
0      솔트룩스  분기보고서 (2024.09)  20241114002412   
1      솔트룩스  반기보고서 (2024.06)  20240814002856   
2      솔트룩스  분기보고서 (2024.03)  20240516001188   
3      솔트룩스  사업보고서 (2023.12)  20240321002058   
4      솔트룩스  분기보고서 (2023.09)  20231114002515   

                                                           URL  
0  https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20241114002412  
1  https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20240814002856  
2  https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20240516001188  
3  https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20240321002058  
4  https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20231114002515  


In [ ]:
import requests
import zipfile
import io

# 🔹 API 정보
api_url = "https://opendart.fss.or.kr/api/document.xml"
api_key = "xxx"  # 여기에 본인의 인증키 입력

# 🔹 요청 파라미터 설정
params = {
    "crtfc_key": api_key,  # API 인증키
    "rcept_no": "20241113000578"  # 예제 접수번호 (실제 검색 후 입력)
}

# 🔹 API 호출
response = requests.get(api_url, params=params)

# 🔹 응답 확인 및 파일 저장
if response.status_code == 200:
    with open("xbrl_report.zip", "wb") as f:
        f.write(response.content)
    print("XBRL ZIP 파일 다운로드 완료: xbrl_report.zip")
else:
    print("API 호출 실패:", response.status_code)


XBRL ZIP 파일 다운로드 완료: xbrl_report.zip


In [ ]:
import requests

api_url = "https://opendart.fss.or.kr/api/document.xml"
api_key = "xxx" 

params = {
    "crtfc_key": api_key,
    "rcept_no": "20190401004781"
}

response = requests.get(api_url, params=params)

print("API 상태 코드:", response.status_code)
print("응답 내용:", response.text[:4000])  # 앞부분 500자만 출력


API 상태 코드: 200
6o�q�۱��͋�+n=M��W�t2u�6���~Ϯ�$��(ƨnHZ�=���|�_N���Ʋd�ts�c���N9i�
�ޑB�|�~������;�1=��O�����K9�{G2���w��p�B�rܑ�"�
����&Rs�h��\ްS���v��137n�L��^'e��ӆ��B/��GN~k�^m]�HGl�Dbg�m��H"�#����
�#}1�t�`�GN.>\�ran�ӥ�N
����90�w�7�6Lt��HG$	�y!
�U���B��5W���}�H�9�v��S���
������pϿ.-�*�;q��J\J�t__d�
�k�D���ބ{��{~��9u�D:c���Ax�} �u�d������~�����s����Fv�nt0���S��� �o
B�`;�6��UQVk�;����5O
����u�	ꑓ��GW_%��D�������$b�p0��ӱ��9�Fx�q:��
����;���r
�������t���������H�wK
��ȥ�՗W��\Z$��C��~�['z��a��݉�I�����O�"(���<�x�x{���gō[��wB�QgḻO
��,�<\LtWw"~����I�u[�;�
xHD�$ 
�� ��F��%Y�#S�d���h�I��_�?o�89
phZ�&��A�N�O����M�W���8r�v���ot+��F�ҿ�e�Zef7n�	;p7I뫹J}�'�az�*TK5���KK�d!!h	!��oy&�Z_��F�;ٶ�-�+;	�[F�z/ĭ�v�Z؄�++�e����nV��b3v u�o!byV~�����	r�vA���}�����k���&�o���������	`��P��{f�N��Ψ�O���?��&(U��;Uf�z�w���r�V_]{0�h����wH�����w7���'�%���>��Wߝ�X�������<�-s���g�SsY}u�a�����_�m�����d�.���z���Í������mڕ�
��1��.��<�
�?�o�

In [63]:
# 네이버 검색 API 예제 - 블로그 검색
import os
import sys
import urllib.request
client_id = os.getenv("NAVER_CLIENT_ID")
client_secret = os.getenv("NAVER_CLIENT_SECRET")
encText = urllib.parse.quote("삼성전자 주가")
url = "https://openapi.naver.com/v1/search/webkr.xml?query=" + encText # JSON 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

<?xml version="1.0" encoding="UTF-8"?><rss version="2.0"><channel><title>Naver Open API - webkr ::&apos;삼성전자 주가&apos;</title><link>https://search.naver.com</link><description>Naver Search Result</description><lastBuildDate>Thu, 20 Feb 2025 16:40:02 +0900</lastBuildDate><total>8282583</total><start>1</start><display>10</display><item><title>&lt;b&gt;삼성전자&lt;/b&gt;: 58,700 -0 (0.00%)</title><link>https://finance.daum.net/quotes/A005930</link><description>전일 : 58,700, 고가 : 58,700, 거래량 : 0(0), 시가 : 58,700, 저가 : 58,700, 거래대금 : 0백만</description></item><item><title>Samsung Electronics Co Ltd 오늘의 &lt;b&gt;주가&lt;/b&gt; | 005930 실시간 티커 - Investing.com</title><link>https://kr.investing.com/equities/samsung-electronics-co-ltd</link><description>&lt;b&gt;삼성전자&lt;/b&gt;의 &lt;b&gt;주가&lt;/b&gt;, 005930 주식, 차트, 기술적 분석, 실적 자료 등 &lt;b&gt;삼성전자&lt;/b&gt; 시세에 대한 자세한 정보를 확인해 보세요. 오늘의 시세뿐만 아니라 뉴스와 분석, 재정 상황, 포럼 등도 확인해 보세요.</description></item><item><title>&lt;b&gt;삼성전자&lt;/b&gt; - 다음 금융</title><link>https://m